In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_log_error, mean_squared_log_error, mean_squared_error, root_mean_squared_error
from xgboost import XGBRegressor
import statsmodels.graphics.tsaplots
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess


In [2]:
# read relevant training data, make copies of the raw data

oil_data_raw = pd.read_csv('data/oil.csv')
holiday_data_raw = pd.read_csv('data/holidays_events.csv')
transaction_data_raw = pd.read_csv('data/transactions.csv')
store_data_raw = pd.read_csv('data/stores.csv')
train_data_raw = pd.read_csv('data/train.csv')

oil_data = oil_data_raw.copy()
holiday_data = holiday_data_raw.copy()
transaction_data = transaction_data_raw.copy()
store_data = store_data_raw.copy()
train_data = train_data_raw.copy()

all_data = [oil_data, holiday_data, transaction_data, store_data, train_data]

In [3]:
# preprocessing: general cleanup

# reformatting dates to datetime objects, drop duplicates

oil_data['date'] = pd.to_datetime(oil_data['date'])
holiday_data['date'] = pd.to_datetime(holiday_data['date'])
transaction_data['date'] = pd.to_datetime(transaction_data['date'])
train_data['date'] = pd.to_datetime(train_data['date'])

oil_data.drop_duplicates(inplace=True)
holiday_data.drop_duplicates(inplace=True)
transaction_data.drop_duplicates(inplace=True)
store_data.drop_duplicates(inplace=True)
train_data.drop_duplicates(inplace=True)

# truncate the data to only include the data before 2017-09-01, as the test data ends at 2017-08-31
oil_data = oil_data[oil_data['date'] < '2017-09-01']
holiday_data = holiday_data[holiday_data['date'] < '2017-09-01']
transaction_data = transaction_data[transaction_data['date'] < '2017-09-01']
train_data = train_data[train_data['date'] < '2017-09-01']

train_data['onpromotion'] = train_data['onpromotion'].astype(int)

In [4]:
# #EDA: check for missing values in each dataset

# for data in all_data:
#     nan_rows_count = data.isnull().any(axis=1).sum()

#     print(f"Number of rows with NaN or null values: {nan_rows_count}")

In [5]:
# #EDA: Oil Price Over Time

# plt.figure(figsize=(10, 5))
# plt.plot(oil_data['date'], oil_data['dcoilwtico'], label='Oil Price')

# plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

# plt.xlabel('Date')
# plt.ylabel('Oil Price')
# plt.title('Oil Price Over Time')
# plt.grid()
# plt.legend()
# plt.gcf().autofmt_xdate()
# plt.show()

In [6]:
#Preprocessing: Fill missing values for oil price with the average of the previous and next values

oil_data['dcoilwtico'] = oil_data['dcoilwtico'].interpolate()


In [7]:
# #EDA: Holiday Events Over Time

# for col in holiday_data.columns:
#     print(f'{col}: {holiday_data[col].unique()}')
#     print(len(holiday_data[col].unique()))

In [8]:
#preprocessing: holiday data

# reduce the number of unique values in the 'type' column 
holiday_data.loc[holiday_data['type'].isin(['Additional', 'Bridge', 'Transfer']), 'type'] = 'Holiday'
holiday_data.loc[holiday_data['transferred'] == True, 'type'] = 'Transferred Holiday'
holiday_data.drop('transferred', axis=1, inplace=True)
holiday_data.drop('description', axis=1, inplace=True)


C:\Users\Doruk\AppData\Local\Temp\ipykernel_18352\159779065.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holiday_data.drop('transferred', axis=1, inplace=True)
C:\Users\Doruk\AppData\Local\Temp\ipykernel_18352\159779065.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holiday_data.drop('description', axis=1, inplace=True)


In [9]:
# #preprocessing: holiday data

# # reduce the number of unique values in the 'description' column, as some values correspond to the same holiday

# holiday_data['description'] = holiday_data['description'].str.replace(r'.*Mundial de futbol Brasil.*', 'World Cup', regex=True)

# start_strings = ['Independencia', 'Fundacion', 'Provincializacion', 'Cantonizacion', 'Terremoto', 'Navidad', 'Dia de la Madre', 'Primer dia del ano']
# useless_strings = ['Traslado ', 'Recupero ', 'Puente ']

# for start_string in start_strings:
#     pattern1 = rf'^{re.escape(start_string)}.*'
#     holiday_data['description'] = holiday_data['description'].str.replace(pattern1, start_string, regex=True)

# for useless_string in useless_strings:
#     pattern2 = rf'{re.escape(useless_string)}.*'
#     holiday_data['description'] = holiday_data['description'].str.replace(pattern2, '', regex=True)

# # Verify the changes
# print(holiday_data['description'].unique())

In [10]:
# #EDA: Transactions Over Time for Each Store

# plt.figure(figsize=(15, 8))

# # Loop through each unique store_nbr
# for store in sorted(transaction_data['store_nbr'].unique()):
#     store_td = transaction_data[transaction_data['store_nbr'] == store]
#     plt.plot(store_td['date'], store_td['transactions'], label=f'Store {store}')

# plt.xlabel('Date')
# plt.ylabel('Transactions')
# plt.title('Transactions Over Time for Each Store')
# plt.grid()
# plt.show()

In [11]:
#Eda: Partial Autocorrelation Plot for Transactions for each store

# for store in sorted(transaction_data['store_nbr'].unique()):
#     store_td = transaction_data[transaction_data['store_nbr'] == store]
#     statsmodels.graphics.tsaplots.plot_pacf(store_td['transactions'], lags=30, title=f'Store {store} PACF')
#     plt.xlabel('Lag')
#     plt.ylabel('Partial Autocorrelation')
#     plt.title(f'Partial Autocorrelation Plot for Transactions for store {store}')
#     plt.grid()
#     plt.show()

In [12]:
#EDA: Train Data, some insights
train_data.set_index('id', inplace=True)

In [13]:
# EDA: Partial Autocorrelation of Sales Data

# this will help us to determine the lag features to include in the model


# for store in sorted(train_data['store_nbr'].unique()):
#     statsmodels.graphics.tsaplots.plot_pacf(train_data[train_data["store_nbr"] == store]['sales'], lags=50)
#     plt.title(f'Partial Autocorrelation of Sales Data for Store {store}')
#     plt.grid()
#     plt.show()

In [14]:
#Preprocessing: Join train_data with transaction_data on 'date' and 'store_nbr'

train_data = train_data.merge(transaction_data[['date', 'store_nbr', 'transactions']], on=['date', 'store_nbr'], how='left')
train_data['transactions'] = train_data['transactions'].fillna(0)

In [15]:
#Preprocessing: Join train_data with oil_data on 'date'

train_data = train_data.merge(oil_data, on='date', how='left')
train_data['dcoilwtico'] = train_data['dcoilwtico'].fillna(93)

In [16]:
#Preprocessing: Join train_data with store_data on 'store_nbr'

train_data = train_data.merge(store_data, on='store_nbr', how='left')

In [17]:
def preprocess_date_info(train_data, holiday_data):
    #Preprocessing: Add column for date information (day, month, year, day of week, is_weekend, is_holiday)

    train_data['day'] = train_data['date'].dt.day
    train_data['month'] = train_data['date'].dt.month
    train_data['season'] = (train_data['month'] % 12 + 3) // 3
    # train_data['season'] = train_data['season'].astype(int)
    train_data['year'] = train_data['date'].dt.year
    # make the dtype of year as int8
    # train_data['year'] = train_data['year'].astype(int)
    train_data['day_of_week'] = train_data['date'].dt.dayofweek
    train_data['is_weekend'] = train_data['day_of_week'].isin([5, 6]).astype(int)

    # add the holiday info as a column, where on a holiday, it is encoded as 1, else 0
    # National > Regional > Local in terms of priority

    train_data['is_holiday'] = 0
    for _, holiday in holiday_data.iterrows():
        if holiday['locale'] == 'Local':
            if holiday['type'] == 'Holiday' or holiday['type'] == 'Transferred Holiday':
                train_data.loc[(train_data['date'] == holiday['date']) & (train_data['city'] == holiday['locale_name']), 'is_holiday'] = 1
            elif holiday['type'] == 'Work Day':
                train_data.loc[(train_data['date'] == holiday['date']) & (train_data['city'] == holiday['locale_name']), 'is_weekend'] = 0

        elif holiday['locale'] == 'Regional':
            if holiday['type'] == 'Holiday' or holiday['type'] == 'Transferred Holiday':
                train_data.loc[(train_data['date'] == holiday['date']) & (train_data['state'] == holiday['locale_name']), 'is_holiday'] = 1
            elif holiday['type'] == 'Work Day':
                train_data.loc[(train_data['date'] == holiday['date']) & (train_data['state'] == holiday['locale_name']), 'is_weekend'] = 0

        elif holiday['locale'] == 'National':
            if holiday['type'] == 'Holiday' or holiday['type'] == 'Transferred Holiday':
                train_data.loc[train_data['date'] == holiday['date'], 'is_holiday'] = 1
            elif holiday['type'] == 'Work Day':
                train_data.loc[train_data['date'] == holiday['date'], 'is_weekend'] = 0           


    # add the payday info as a column, where on 15th and last day of the month, it is payday, encoded as 1, else 0

    train_data['is_payday'] = 0
    train_data.loc[(train_data['day'] == 15) | (train_data['day'] == train_data['date'].dt.days_in_month), 'is_payday'] = 1

    # drop date
    # train_data.drop('date', axis=1, inplace=True)

    return train_data

train_data = preprocess_date_info(train_data, holiday_data)

In [18]:
fourier_m = CalendarFourier(freq='ME', order=4)
dp = DeterministicProcess(
    index=train_data.date,
    constant=True,
    order=0,  # Skip the linear trend if you already have time encodings
    seasonal=False,  # Skip weekly seasonality since you already have day_of_week and is_weekend
    additional_terms=[fourier_m],  # Only adding monthly Fourier terms for smooth seasonality
    drop=True
)

In [19]:
a = dp.in_sample()

In [20]:
#add the deterministic process to the train_data
train_data.set_index('date', inplace=True)

train_data = pd.concat([train_data, a], axis=1)

train_data.reset_index(inplace=True)


In [21]:
# # Feature Engineering: Create Lag Features for target variable 'sales'

# for lag in range(1, 17):
#     train_data[f'sales_lag_{lag}'] = train_data.groupby('store_nbr')['sales'].shift(lag)
#     # fill the NaN values with 0
#     train_data[f'sales_lag_{lag}'] = train_data[f'sales_lag_{lag}'].fillna(0)

# # TODO: filter out the rows that correspond to the first 16 days of each store
# train_data.dropna(inplace=True)

In [22]:
# Feature Engineering: Encode Categorical Variables

#categorical_cols = ['store_nbr', 'family', 'city', 'state', 'type', 'cluster', 'day', 'month', 'season', 'day_of_week']
categorical_cols = ['family', 'city', 'state', 'type', 'cluster', 'day', 'month', 'season', 'day_of_week']

train_data = pd.get_dummies(train_data, columns=categorical_cols, dtype=int)


In [23]:
# define the target variable and features
# target values are all the columns starting with 'sales_lag_' of which there are 16

# target = train_data.filter(regex='sales_lag_')
# features = train_data.drop(['date', 'sales'] + target.columns.tolist(), axis=1)

target = train_data["sales"]
features = train_data.drop(["sales", "date"], axis=1)

# split the data into training and validation sets

split_index = int(len(features) * 0.9)
X_train, X_val = features.iloc[:split_index], features.iloc[split_index:]
y_train, y_val = target.iloc[:split_index], target.iloc[split_index:]


# verify the shapes of the training and validation sets
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(2700799, 164) (300089, 164) (2700799,) (300089,)


In [24]:
#Training The Model: The Linear Regression Model

lr = ElasticNet()
lr.fit(X_train, y_train)

ElasticNet()

In [25]:
y_train_pred_lr = lr.predict(X_train)
y_val_pred_lr = lr.predict(X_val)

In [26]:
root_mean_squared_error_train_lr = root_mean_squared_error(y_train, y_train_pred_lr)
root_mean_squared_error_val_lr = root_mean_squared_error(y_val, y_val_pred_lr)

In [27]:
#Training The Model: Residuals

residuals = y_train - y_train_pred_lr

# Train the XGBoost model on the residuals
xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
xgb_model.fit(X_train, residuals)

# Predict from the combined model on the validation set

y_pred_lr_val = lr.predict(X_val)
y_pred_xgb_val = xgb_model.predict(X_val)

y_pred_val = y_pred_lr_val + y_pred_xgb_val

# # replace negative values with 0
# y_pred_val = np.where(y_pred_val < 0, 0, y_pred_val)

In [28]:
# Calculate the RMSLE on the validation set

root_mean_squared_error(y_val, y_pred_val)

330.82406028850363

In [152]:
# Make predictions on the test set, and save the results to a CSV file

# read the test data
test_data = pd.read_csv('data/test.csv')

test_data.info()

# preprocess the test data

test_data['date'] = pd.to_datetime(test_data['date'])
test_data['onpromotion'] = test_data['onpromotion'].astype(int)

# create a copy of transaction_data
transaction_data_test = transaction_data.copy()
transaction_data_test['date_one_year_later'] = transaction_data_test['date'] + pd.DateOffset(years=1)
test_data = test_data.merge(transaction_data_test, left_on=['date', "store_nbr"], right_on=['date_one_year_later', "store_nbr"], how='left')
test_data.drop(columns=['date_y','date_one_year_later'], inplace=True)
test_data.rename(columns={'date_x': 'date'}, inplace=True)

# join the test data with the oil data, store data, and holiday data

test_data = test_data.merge(oil_data, on='date', how='left')
# test_data['dcoilwtico'] = test_data['dcoilwtico'].fillna(93)
test_data = test_data.merge(store_data, on='store_nbr', how='left')
test_data = preprocess_date_info(test_data, holiday_data)

#set id column as index, replacing the default index
test_data.set_index('id', inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB


In [153]:
# Generate out-of-sample forecasts for the next 16 steps
forecast_steps = 16
forecast_dates = pd.date_range(start=train_data['date'].max() + pd.Timedelta(days=1), periods=forecast_steps, freq='D')

forecast_values = dp.out_of_sample(steps=forecast_steps, forecast_index=forecast_dates)
forecast_values["date"] =  forecast_dates

test_data = test_data.merge(forecast_values, on='date', how='left')

# encode the categorical variables
test_data = pd.get_dummies(test_data, columns=categorical_cols, dtype=int)
test_data = test_data.drop('date', axis=1)

In [154]:
true_col=[]
dummy_col= []

for col in X_val.columns:
    true_col.append(col)
    dummy_col.append(col)

print(len(dummy_col),dummy_col)
    

164 ['store_nbr', 'onpromotion', 'transactions', 'dcoilwtico', 'year', 'is_weekend', 'is_holiday', 'is_payday', 'const', 'sin(1,freq=ME)', 'cos(1,freq=ME)', 'sin(2,freq=ME)', 'cos(2,freq=ME)', 'sin(3,freq=ME)', 'cos(3,freq=ME)', 'sin(4,freq=ME)', 'cos(4,freq=ME)', 'family_AUTOMOTIVE', 'family_BABY CARE', 'family_BEAUTY', 'family_BEVERAGES', 'family_BOOKS', 'family_BREAD/BAKERY', 'family_CELEBRATION', 'family_CLEANING', 'family_DAIRY', 'family_DELI', 'family_EGGS', 'family_FROZEN FOODS', 'family_GROCERY I', 'family_GROCERY II', 'family_HARDWARE', 'family_HOME AND KITCHEN I', 'family_HOME AND KITCHEN II', 'family_HOME APPLIANCES', 'family_HOME CARE', 'family_LADIESWEAR', 'family_LAWN AND GARDEN', 'family_LINGERIE', 'family_LIQUOR,WINE,BEER', 'family_MAGAZINES', 'family_MEATS', 'family_PERSONAL CARE', 'family_PET SUPPLIES', 'family_PLAYERS AND ELECTRONICS', 'family_POULTRY', 'family_PREPARED FOODS', 'family_PRODUCE', 'family_SCHOOL AND OFFICE SUPPLIES', 'family_SEAFOOD', 'city_Ambato', 'c

In [155]:
dummy_col_test = []
for col in test_data.columns:
    dummy_col_test.append(col)

print(len(dummy_col_test), dummy_col_test)

135 ['store_nbr', 'onpromotion', 'transactions', 'dcoilwtico', 'year', 'is_weekend', 'is_holiday', 'is_payday', 'const', 'sin(1,freq=ME)', 'cos(1,freq=ME)', 'sin(2,freq=ME)', 'cos(2,freq=ME)', 'sin(3,freq=ME)', 'cos(3,freq=ME)', 'sin(4,freq=ME)', 'cos(4,freq=ME)', 'family_AUTOMOTIVE', 'family_BABY CARE', 'family_BEAUTY', 'family_BEVERAGES', 'family_BOOKS', 'family_BREAD/BAKERY', 'family_CELEBRATION', 'family_CLEANING', 'family_DAIRY', 'family_DELI', 'family_EGGS', 'family_FROZEN FOODS', 'family_GROCERY I', 'family_GROCERY II', 'family_HARDWARE', 'family_HOME AND KITCHEN I', 'family_HOME AND KITCHEN II', 'family_HOME APPLIANCES', 'family_HOME CARE', 'family_LADIESWEAR', 'family_LAWN AND GARDEN', 'family_LINGERIE', 'family_LIQUOR,WINE,BEER', 'family_MAGAZINES', 'family_MEATS', 'family_PERSONAL CARE', 'family_PET SUPPLIES', 'family_PLAYERS AND ELECTRONICS', 'family_POULTRY', 'family_PREPARED FOODS', 'family_PRODUCE', 'family_SCHOOL AND OFFICE SUPPLIES', 'family_SEAFOOD', 'city_Ambato', 'c

In [156]:
# remove the intersection of columns between dummy_col and dummy_col_test from both lists

common_cols = list(set(dummy_col).intersection(dummy_col_test))

for col in common_cols:
    dummy_col.remove(col)
    dummy_col_test.remove(col)


In [157]:
print(len(dummy_col),dummy_col)
print(len(dummy_col_test), dummy_col_test)

29 ['day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'day_7', 'day_8', 'day_9', 'day_10', 'day_11', 'day_12', 'day_13', 'day_14', 'day_15', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_9', 'month_10', 'month_11', 'month_12', 'season_1', 'season_2', 'season_4']
0 []


In [158]:
# add the missing columns to the test data, filled with 0
for col in dummy_col:
    test_data[col] = 0

In [159]:
test_data.tail()

,store_nbr,onpromotion,transactions,dcoilwtico,year,is_weekend,is_holiday,is_payday,const,"sin(1,freq=ME)",...,month_5,month_6,month_7,month_9,month_10,month_11,month_12,season_1,season_2,season_4
28507,9,1,1954.0,47.26,2017,0,0,1,1.0,-0.201299,...,0,0,0,0,0,0,0,0,0,0
28508,9,0,1954.0,47.26,2017,0,0,1,1.0,-0.201299,...,0,0,0,0,0,0,0,0,0,0
28509,9,1,1954.0,47.26,2017,0,0,1,1.0,-0.201299,...,0,0,0,0,0,0,0,0,0,0
28510,9,9,1954.0,47.26,2017,0,0,1,1.0,-0.201299,...,0,0,0,0,0,0,0,0,0,0
28511,9,0,1954.0,47.26,2017,0,0,1,1.0,-0.201299,...,0,0,0,0,0,0,0,0,0,0


In [160]:
# reorder the columns of test_data to match the training data
test_data = test_data[true_col]

In [161]:
test_data["dcoilwtico"] = test_data["dcoilwtico"].interpolate()
test_data["transactions"] = test_data["transactions"].interpolate()

In [162]:
# predict the sales for the test data
y_test_lr_pred = lr.predict(test_data)
y_test_xgb_pred = xgb_model.predict(test_data)

y_test_pred = y_test_lr_pred + y_test_xgb_pred

# replace the sales column of 'sample_submission.csv' with the predicted values, and save the results to the same CSV file

sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission['sales'] = y_test_pred
sample_submission.to_csv('data/sample_submission.csv', index=False)
